In [1]:
import json
from os.path import join, dirname
from os import listdir
import re
import pandas as pd

from watson_developer_cloud import DocumentConversionV1
from watson_developer_cloud import AlchemyLanguageV1

In [19]:
doc = "Dove.docx"
tmp = !pwd
dir = tmp[0] 

formats = ['doc','docx','pdf']

In [10]:
credentials = {
    "url": "https://gateway.watsonplatform.net/document-conversion/api",
    "password": "tQDOCgBzcC0K",
    "username": "1a238d8d-f4d7-45eb-a5b5-2e1ff2ba8797",
}

api_key = "568d7754d14e108c87191b14bb23bec6623eee8a"

In [11]:
document_conversion = DocumentConversionV1(
    username=credentials.get('username'),
    password=credentials.get('password'),
    version='2016-06-30')

In [12]:
alchemy_language = AlchemyLanguageV1(api_key=api_key)
combined_operations = ['page-image', 'entity', 'keyword', 'title', 'author', 'taxonomy', 'concept', 'doc-emotion','quantity']

In [13]:
def xConvert(doc):

    with open(join(dir, doc), 'rb') as document:
        config = {'conversion_target': DocumentConversionV1.ANSWER_UNITS}
        output = document_conversion.convert_document(document=document, config=config)
        #print(json.dumps(output, indent=2))
    
    return output

output = xConvert(doc)

In [30]:
def xFilter(output):

    blocks = []
    for au in output.get('answer_units'):
        au_text = au.get('content')[0].get('text')
        if "viewab" in au_text.lower():
            #print(au_text)
            #print('\n')
            blocks.append(au_text)
    
    return blocks

blocks = xFilter(output)

TypeError: 'NoneType' object is not iterable

In [35]:
table = pd.DataFrame(columns=['document','valid','left context','quantity','right context','snippet'])
i = 0

for doc in listdir(dir):
    
    ext = doc.split(".")[-1]
    if ext not in formats:
        print("Unsupported format: %s" % (doc))
        continue
    
    output = xConvert(doc)
    blocks = xFilter(output)
    
    for block in blocks:
        output = alchemy_language.combined(text=block, extract=combined_operations)
        for entity in output.get('entities'):
            if entity.get('type') == 'Quantity':
                covered_text = entity.get('text')
                left_context = ''
                right_context = ''
                for m in re.finditer(covered_text, block):
                    start = m.start()
                    end = m.end()
                    if start >= 50:
                        left_context = block[start-50:start]
                    else:
                        left_context = block[:start]
                    end = start + len(covered_text)
                    if len(block) - end >= 50:
                        right_context = block[end:end+50]
                    else:
                        right_context = block[end:] 
                i = i + 1
                table.loc[i] = [doc,'',left_context, covered_text, right_context,block]
        
table.to_csv(join(dir, 'table.csv'))

Unsupported format: .DS_Store
Unsupported format: .git
Unsupported format: .ipynb_checkpoints
Unsupported format: python_environment.yml
Unsupported format: RFP.ipynb
Unsupported format: table.csv


In [36]:
table.head(10)

,document,valid,left context,quantity,right context,snippet
1,052516_Smell Em Whos Boss_Old Spice RFP.pdf,,measure of success included • We plan to ha...,100K,product sample sizes to distribute and are lo...,"THE ASSIGNMENT On July 11th, Old Spice will be..."
2,052516_Smell Em Whos Boss_Old Spice RFP.pdf,,High SOV on all recommended placements (i.e....,20%,) • We will be tracking viewability and aim ...,"THE ASSIGNMENT On July 11th, Old Spice will be..."
3,052516_Smell Em Whos Boss_Old Spice RFP.pdf,,ng video presence - plans should include at le...,80%,of budget against video inventory. Mobile vid...,"THE ASSIGNMENT On July 11th, Old Spice will be..."
4,2016 Glenmorangie Digital RFP_FINAL.docx,,"Jersey 6% (201), Massachusetts 6% (261), Flor...",6%,"(84) Attitude/Interest Business / finance, co...",About the Brand Glenmorangie is a highland sin...
5,2016 Glenmorangie Digital RFP_FINAL.docx,,"20% (121), 40-49 24% (128), 50-59 17% (98) - ...",75K,+ (average $145K) - Caucasian significantly ov...,About the Brand Glenmorangie is a highland sin...
6,2016 Glenmorangie Digital RFP_FINAL.docx,,"an & suburban centers California 14% (118), Te...",9%,"(120), New York 7% (109), New Jersey 6% (201)...",About the Brand Glenmorangie is a highland sin...
7,2016 Glenmorangie Digital RFP_FINAL.docx,,itioning/creative idea has been expressed in p...,four years,"through our print, TV, and more recently, dig...",About the Brand Glenmorangie is a highland sin...
8,2016 Glenmorangie Digital RFP_FINAL.docx,,at least 50% of the ad was viewable for at le...,one second,CTR Blocking percentage Completion Rate (vide...,About the Brand Glenmorangie is a highland sin...
9,2016 Glenmorangie Digital RFP_FINAL.docx,,"are new to the category, have a younger profil...",35 years,old). Malt whisky is one of the tools used to...,About the Brand Glenmorangie is a highland sin...
10,2016 Glenmorangie Digital RFP_FINAL.docx,,"f the perfect finishes, off-premise activation...",2 hour,media appearance (still TBD) as well as 10 so...,About the Brand Glenmorangie is a highland sin...
